Description:

Input:
This file takes as input geojson OSM dataframes containing the polygons of:

- service stations
- car parking areas close to service stations
- truck parking areas close to service stations

The dataframes are transformed by deleting non-polygon rows.

The car and truck parking dataframes are merged into one, containing all parking areas at service stations: all_parking

Output:
- all_parking (1606, 6): geojson dataframe containing all parking (car and truck) polygons at service stations and service station ids
- rest_stations (506, 5): geojson dataframe containing all service stations polygons

Of the 506 service stations with polygons, 467 contain annotated parking spaces.


# Libraries and Google Drive

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon

import os

In [2]:
# Mount Drive
#from google.colab import drive
#drive.mount("/content/drive")

In [3]:
# Set wd
os.chdir('/Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas')

print("Working directory:", os.getcwd())

Working directory: /Users/benediktkorbach/Documents/GitHub/remote-sensing-of-parking-areas


# Import OSM data and combine into one dataframe

In [4]:
# Load the OSM GeoJSON files containing rest station area polygons and car/ truck parking area polygons
rest_stations = gpd.read_file("02_data_acquisition/OSM_exports/OSM_service_stops.geojson")
car_parkings = gpd.read_file("02_data_acquisition/OSM_exports/OSM_car_parking_service_stops.geojson")
truck_parkings = gpd.read_file("02_data_acquisition/OSM_exports/OSM_truck_parking_service_stops.geojson")

## Inspect and Transform JSONs

In [5]:
# Set the option to display all columns
pd.set_option("display.max_columns", None)

In [6]:
# Print shapes of geopandas Dataframe
print("Shape rest_stations:", rest_stations.shape)
print("Shape car_stations:", car_parkings.shape)
print("Shape truck_parkings:", truck_parkings.shape)

Shape rest_stations: (510, 65)
Shape car_stations: (2299, 112)
Shape truck_parkings: (1043, 104)


In [7]:
# Functions used to transform dataframes

def keep_polygon_rows(gdf):
    """
    Filter out rows that contain polygons or multipolygons.

    Parameters:
    gdf (geopandas geodataframe): geodataframe containing geometry column

    Returns:
    A geodataframe consisting of rows with polygons/ multipolygons.
    """

    # Define a function to check the geometry type
    def is_polygon(geometry):
        return isinstance(geometry, (Polygon, MultiPolygon))

    # Apply the function to the 'geometry' column and filter the df
    gdf_filtered = gdf[gdf["geometry"].apply(is_polygon)]

    return gdf_filtered


def create_id(row):
    """
    Create unique identifier for each rest stop.

    Parameters:
    row: row of a geopandas geodataframe

    Returns:
    String containing id.
    """
    coord = None  # Initialize coord with a default value
    try:
        if row["geometry"].geom_type == "Polygon":
            coord = row["geometry"].exterior.coords[0]
        elif row["geometry"].geom_type == "MultiPolygon":
            coord = list(row["geometry"].geoms)[0].exterior.coords[0] # inspect first polygon in multipolygon

        lon = round(coord[0], 8)
        lat = round(coord[1], 8)
        id = f"lon_{lon}_lat_{lat}"

        return id

    except Exception as e:
        print(f"Error occurred: {e}")

        return None

### Inspect and transform rest_stations

In [8]:
type(rest_stations)

geopandas.geodataframe.GeoDataFrame

In [9]:
rest_stations.head()

,id,@id,access,aeroway,alt_name,alt_name:de,alt_name:nl,amenity,area,atmotorway,bin,capacity,capacity:abnormal_load,capacity:bus,capacity:car,capacity:caravan,capacity:disabled,capacity:hgv,capacity:long,changing_table,check_date,check_date:toilets:wheelchair,description,direction,drinking_water,fee,fixme,hazmat:water,hgv,highway,landuse,lit,mapillary,motorcar,name,name:alt,name:de,name:dsb,name:etymology:wikidata,noise_barrier,note,note:de,old_name,opening_date,opening_hours,operator,operator:wikidata,park_ride,parking,picnic_table,preserved,shower,source,start_date,supervised,surface,survey:date,toilets,toilets:fee,toilets:wheelchair,type,wheelchair,wikidata,wikipedia,geometry
0,relation/3947321,relation/3947321,yes,None,None,None,None,parking,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,no,None,None,designated,rest_area,None,None,None,None,Hohe Schrecke West,None,None,None,None,None,None,None,None,None,None,None,None,None,surface,None,None,None,None,None,None,paving_stones,None,None,None,None,multipolygon,None,None,None,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,relation/4672225,relation/4672225,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,rest_area,None,None,None,None,Probstheide,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,multipolygon,yes,None,None,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,relation/4672228,relation/4672228,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,rest_area,None,None,None,None,Ladeburger Heide,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,multipolygon,yes,None,None,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,way/22568867,way/22568867,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,rest_area,None,None,None,None,Schäferborn,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,way/22568868,way/22568868,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,rest_area,None,None,None,None,Spießwald,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,yes,None,None,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


In [11]:
# Transform rest_stations to only contain relevant columns

# Drop non-polygon rows
rest_stations = keep_polygon_rows(rest_stations)
print("Shape rest_stations:", rest_stations.shape)

# Keep necessary columns
keep_columns = ["id", "@id", "highway", "name", "geometry"]
rest_stations = rest_stations[keep_columns]

Shape rest_stations: (506, 65)


In [12]:
rest_stations.head()

,id,@id,highway,name,geometry
0,relation/3947321,relation/3947321,rest_area,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,relation/4672225,relation/4672225,rest_area,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,relation/4672228,relation/4672228,rest_area,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,way/22568867,way/22568867,rest_area,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,way/22568868,way/22568868,rest_area,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


In [13]:
rest_stations['id'] = rest_stations.apply(create_id, axis=1)

In [14]:
rest_stations.head()

,id,@id,highway,name,geometry
0,lon_11.2442867_lat_51.3248863,relation/3947321,rest_area,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,relation/4672225,rest_area,Probstheide,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,relation/4672228,rest_area,Ladeburger Heide,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6630029_lat_50.2542348,way/22568867,rest_area,Schäferborn,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,lon_8.6624068_lat_50.2544468,way/22568868,rest_area,Spießwald,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


In [15]:
rest_stations_geoms = rest_stations[["id", "geometry"]]

In [16]:
rest_stations_geoms.head()

,id,geometry
0,lon_11.2442867_lat_51.3248863,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,lon_13.5458137_lat_52.7129474,"POLYGON ((13.54581 52.71295, 13.54556 52.71258..."
2,lon_13.549042_lat_52.7092396,"POLYGON ((13.54904 52.70924, 13.54912 52.70919..."
3,lon_8.6630029_lat_50.2542348,"POLYGON ((8.66300 50.25423, 8.66302 50.25237, ..."
4,lon_8.6624068_lat_50.2544468,"POLYGON ((8.66241 50.25445, 8.66205 50.25369, ..."


### Inspect and transform truck_parkings

In [18]:
type(truck_parkings)

geopandas.geodataframe.GeoDataFrame

In [21]:
truck_parkings.head()

,id,@id,TMC:cid_58:tabcd_1:Class,TMC:cid_58:tabcd_1:Direction,TMC:cid_58:tabcd_1:LCLversion,TMC:cid_58:tabcd_1:LocationCode,TMC:cid_58:tabcd_1:NextLocationCode,TMC:cid_58:tabcd_1:PrevLocationCode,abnormal_load,access,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,amenity,amenity:truck_stop,area,atmotorway,bdouble,bureau_de_change,bus,capacity,capacity:abnormal_load,capacity:bus,capacity:car,capacity:caravan,capacity:disabled,capacity:hgv,capacity:long,capacity:motorcar,capacity:parent,capacity:restaurant,capacity:shower,capacity:truck,capacity:truck_diesel_pumps,capacity:washers,capacity:women,car,caravan,check_date:fee,coach,contact:email,contact:fax,contact:phone,contact:website,copier,covered,description,description:en,fast_food,fax,fee,goods,hgv,highway,internet_access,lit,maxheight,maxstay,motor_vehicle,motorcar,motorcycle,motorhome,name,name:de,note,opening_hours,operator,orientation,park_ride,parking,parking:condition:vehicles,parking:orientation,parking_space,payment:american_express,payment:bancomat,payment:diners_club,payment:dkv,payment:ec,payment:esso_card,payment:euroshell,payment:mastercard,payment:uta,payment:visa,playground,psv,restaurant,shop,smoothness,source,supervised,surface,@relations,toilets,toilets:wheelchair,tourist_bus,trailer,truck_repair,tv,type,vehicle,wheelchair,geometry
0,relation/3947321,relation/3947321,None,None,None,None,None,None,None,yes,None,None,None,None,None,parking,None,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,no,None,designated,rest_area,None,None,None,None,None,None,None,None,Hohe Schrecke West,None,None,None,None,None,None,surface,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,paving_stones,None,None,None,None,None,None,None,multipolygon,None,None,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,relation/11487370,relation/11487370,None,None,None,None,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,designated,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,"POLYGON ((9.51885 51.26893, 9.51880 51.26888, ..."
2,relation/12993807,relation/12993807,None,None,None,None,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,no,None,designated,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,surface,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,yes,"MULTIPOLYGON (((9.54446 54.47736, 9.54388 54.4..."
3,relation/13277585,relation/13277585,None,None,None,None,None,None,None,yes,None,None,None,None,None,parking,None,None,None,None,None,None,56,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,no,None,designated,None,None,None,None,None,None,no,None,None,None,None,None,None,None,None,None,surface,None,None,None,None,None,None,None,None,None,None,None,None,None,None,no,None,None,None,None,None,paving_stones,None,None,None,None,None,None,None,multipolygon,None,None,"MULTIPOLYGON (((11.42584 51.43480, 11.42585 51..."
4,relation/13277586,relation/13277586,None,None,None,None,None,None,None,yes,None,None,None,None,None,parking,None,None,None,None,None,None,56,None,None,None,None,None,None,None,None,

In [17]:
# Transform truck_parkings to only contain relevant columns

# Delete non-polygon rows
truck_parkings = keep_polygon_rows(truck_parkings)

# Keep necessary columns
keep_columns = ["id", "@id", "hgv", "name", "geometry"]
truck_parkings = truck_parkings[keep_columns]
print("Shape truck_parkings:", truck_parkings.shape)

Shape truck_parkings: (1028, 5)


In [18]:
truck_parkings.head()

,id,@id,hgv,name,geometry
0,relation/3947321,relation/3947321,designated,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51..."
1,relation/11487370,relation/11487370,designated,None,"POLYGON ((9.51885 51.26893, 9.51880 51.26888, ..."
2,relation/12993807,relation/12993807,designated,None,"MULTIPOLYGON (((9.54446 54.47736, 9.54388 54.4..."
3,relation/13277585,relation/13277585,designated,None,"MULTIPOLYGON (((11.42584 51.43480, 11.42585 51..."
4,relation/13277586,relation/13277586,designated,None,"MULTIPOLYGON (((11.42634 51.43379, 11.42626 51..."


### Inspect and transform car_parkings

In [19]:
type(car_parkings)

geopandas.geodataframe.GeoDataFrame

In [20]:
car_parkings.head()

,id,@id,access,access:description,access:disabled,access:hgv,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,addr:suburb,amenity,area,atmotorway,barrier,building,bus,capacity,capacity:abnormal_load,capacity:bus,capacity:car,capacity:caravan,capacity:charging,capacity:disabled,capacity:hgv,capacity:hgv:abnormal_load,capacity:motorcar,capacity:motorcar_trailer,capacity:parent,capacity:restaurant,capacity:shower,capacity:smoker_free,capacity:toilets,capacity:tourist_bus,capacity:truck,capacity:truck_diesel_pumps,capacity:washers,capacity:women,car,caravan,caravans,check_date,check_date:fee,coach,contact:fax,contact:phone,copier,description,disabled,disabled_spaces,ele,fast_food,fee,fixme,goods,hgv,hgv:conditional,highway,internet_access,landuse,layer,lit,maxheight,maxstay,motor_vehicle,motorcar,motorcar:conditional,motorcycle,motorcycle:conditional,motorhome,name,name:alt,name:ru,note,opening_hours,operator,operator:type,orientation,park_ride,parking,parking:condition:vehicles,parking:hgv,parking:orientation,parking_space,payment:dkv,payment:ec,payment:mastercard,payment:uta,payment:visa,psv,restaurant,shop,smoothness,source,supervised,surface,@relations,survey:date,toilets,toilets:wheelchair,tourism,tourist_bus,traffic_sign,trailer,truck,tv,type,vehicle,wheelchair,wheelchair:description,geometry
0,relation/11487080,relation/11487080,None,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,street_side,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,None,"POLYGON ((9.52106 51.26784, 9.52101 51.26783, ..."
1,relation/11489034,relation/11489034,private,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,surface,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,None,"POLYGON ((9.52541 51.26901, 9.52542 51.26898, ..."
2,relation/11489035,relation/11489035,None,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,None,"POLYGON ((9.52566 51.27037, 9.52586 51.26995, ..."
3,relation/11489084,relation/11489084,customers,None,None,None,None,None,None,None,None,None,parking,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,surface,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,multipolygon,None,None,None,"POLYGON ((9.52691 51.27054, 9.52698 51.27045, ..."
4,relation/13277584,relation/13277584,yes,None,None,None,None,

In [21]:
# Transform car_parkings to only contain relevant columns

# Delete non-polygon rows
car_parkings = keep_polygon_rows(car_parkings)

# Keep necessary columns
keep_columns = ["id", "@id", "name", "geometry"]
car_parkings = car_parkings[keep_columns]
print("Shape car_parkings:", car_parkings.shape)

Shape car_parkings: (2194, 4)


In [22]:
car_parkings.head()

,id,@id,name,geometry
0,relation/11487080,relation/11487080,None,"POLYGON ((9.52106 51.26784, 9.52101 51.26783, ..."
1,relation/11489034,relation/11489034,None,"POLYGON ((9.52541 51.26901, 9.52542 51.26898, ..."
2,relation/11489035,relation/11489035,None,"POLYGON ((9.52566 51.27037, 9.52586 51.26995, ..."
3,relation/11489084,relation/11489084,None,"POLYGON ((9.52691 51.27054, 9.52698 51.27045, ..."
4,relation/13277584,relation/13277584,None,"MULTIPOLYGON (((11.42781 51.43533, 11.42764 51..."


## Merge DataFrames

In [49]:
# Print shapes of geopandas Dataframe after removing rows not containing polygons
print("Shape rest_stations:", rest_stations.shape)
print("Shape car_stations:", car_parkings.shape)
print("Shape truck_parkings:", truck_parkings.shape)

Shape rest_stations: (506, 5)
Shape car_stations: (2194, 4)
Shape truck_parkings: (1028, 5)


In [50]:
# Combine GeoDataFrames into one

# Perform spatial join - parking areas to rest stations
car_parkings_joined = gpd.sjoin(car_parkings, rest_stations_geoms, how="left", predicate='intersects', lsuffix = "car", rsuffix = "rest")
truck_parkings_joined = gpd.sjoin(truck_parkings, rest_stations_geoms, how="left", predicate='intersects', lsuffix = "truck", rsuffix = "rest")

In [51]:
car_parkings_joined.head()

,id_car,@id,name,geometry,index_rest,id_rest
0,relation/11487080,relation/11487080,None,"POLYGON ((9.52106 51.26784, 9.52101 51.26783, ...",NaN,NaN
1,relation/11489034,relation/11489034,None,"POLYGON ((9.52541 51.26901, 9.52542 51.26898, ...",NaN,NaN
2,relation/11489035,relation/11489035,None,"POLYGON ((9.52566 51.27037, 9.52586 51.26995, ...",NaN,NaN
3,relation/11489084,relation/11489084,None,"POLYGON ((9.52691 51.27054, 9.52698 51.27045, ...",NaN,NaN
4,relation/13277584,relation/13277584,None,"MULTIPOLYGON (((11.42781 51.43533, 11.42764 51...",NaN,NaN


In [52]:
car_parkings_joined.shape

(2194, 6)

In [53]:
# Filter out rows that do not correspond to a rest/ service stop

car_parkings_joined = car_parkings_joined[car_parkings_joined["id_rest"].notnull()]
car_parkings_joined = car_parkings_joined.drop("index_rest", axis=1)

In [54]:
car_parkings_joined.head()

,id_car,@id,name,geometry,id_rest
6,relation/13277622,relation/13277622,None,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53...",lon_10.2703215_lat_53.3301407
7,relation/13277623,relation/13277623,None,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53...",lon_10.2712414_lat_53.331074
8,relation/15482166,relation/15482166,None,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52...",lon_12.140788_lat_52.2431929
9,relation/15482306,relation/15482306,None,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52...",lon_12.149456_lat_52.2449316
23,way/31129920,way/31129920,Nord-Ostsee-Kanal,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ...",lon_9.323161_lat_54.0686721


In [55]:
car_parkings_joined.shape

(999, 5)

In [56]:
truck_parkings_joined.head()

,id_truck,@id,hgv,name,geometry,index_rest,id_rest
0,relation/3947321,relation/3947321,designated,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51...",227.0,lon_11.2429922_lat_51.3232808
0,relation/3947321,relation/3947321,designated,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51...",0.0,lon_11.2442867_lat_51.3248863
1,relation/11487370,relation/11487370,designated,None,"POLYGON ((9.51885 51.26893, 9.51880 51.26888, ...",NaN,NaN
2,relation/12993807,relation/12993807,designated,None,"MULTIPOLYGON (((9.54446 54.47736, 9.54388 54.4...",NaN,NaN
3,relation/13277585,relation/13277585,designated,None,"MULTIPOLYGON (((11.42584 51.43480, 11.42585 51...",NaN,NaN


In [57]:
truck_parkings_joined.shape

(1030, 7)

In [58]:
truck_parkings_joined = truck_parkings_joined[truck_parkings_joined["id_rest"].notnull()]
truck_parkings_joined = truck_parkings_joined.drop("index_rest", axis=1)

In [59]:
truck_parkings_joined.head()

,id_truck,@id,hgv,name,geometry,id_rest
0,relation/3947321,relation/3947321,designated,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51...",lon_11.2429922_lat_51.3232808
0,relation/3947321,relation/3947321,designated,Hohe Schrecke West,"MULTIPOLYGON (((11.24429 51.32489, 11.24430 51...",lon_11.2442867_lat_51.3248863
5,way/4701435,way/4701435,designated,None,"POLYGON ((9.54079 50.93981, 9.54063 50.94007, ...",lon_9.5406188_lat_50.9403942
8,way/24301057,way/24301057,designated,None,"POLYGON ((13.79667 52.43752, 13.79614 52.43743...",lon_13.7976643_lat_52.4383976
9,way/24301220,way/24301220,designated,Kalkberge,"POLYGON ((13.79893 52.45696, 13.79870 52.45678...",lon_13.7994366_lat_52.4570641


In [60]:
truck_parkings_joined.shape

(607, 6)

In [61]:
car_parkings_joined['type'] = 'car'
truck_parkings_joined['type'] = 'truck'

# Combine data
all_parkings = gpd.GeoDataFrame(pd.concat([car_parkings_joined, truck_parkings_joined], ignore_index=True))

In [62]:
all_parkings = all_parkings.drop(["id_truck", "hgv"], axis=1)

In [63]:
all_parkings.head()

,id_car,@id,name,geometry,id_rest,type
0,relation/13277622,relation/13277622,None,"MULTIPOLYGON (((10.26903 53.33039, 10.26926 53...",lon_10.2703215_lat_53.3301407,car
1,relation/13277623,relation/13277623,None,"MULTIPOLYGON (((10.26848 53.33228, 10.26872 53...",lon_10.2712414_lat_53.331074,car
2,relation/15482166,relation/15482166,None,"MULTIPOLYGON (((12.14216 52.24271, 12.14213 52...",lon_12.140788_lat_52.2431929,car
3,relation/15482306,relation/15482306,None,"MULTIPOLYGON (((12.14713 52.24497, 12.14717 52...",lon_12.149456_lat_52.2449316,car
4,way/31129920,way/31129920,Nord-Ostsee-Kanal,"POLYGON ((9.32127 54.06898, 9.32314 54.06861, ...",lon_9.323161_lat_54.0686721,car


In [65]:
unique_entries_count = all_parkings['id_rest'].nunique()

print("Number of unique rest_stops:", unique_entries_count)

Number of unique rest_stops: 467


In [66]:
# Print shapes of geopandas Dataframe after merging
print("Shape rest_stations:", rest_stations.shape)
print("Shape all_parkings:", all_parkings.shape)

Shape rest_stations: (506, 5)
Shape all_parkings: (1606, 6)


## Inspect all_parkings

In [76]:
unique_rest_stops = all_parkings['id_rest'].unique()
print("Unique rest stops:", len(unique_rest_stops))

Unique rest stops: 467


In [43]:
# Create dictionary that indicates the number of car/truck polygons per rest stops

rest_stop_dic = {}

for index, row in all_parkings.iterrows():
    id_rest = row["id_rest"]
    car_count = 0
    truck_count = 0

    if id_rest in rest_stop_dic:
        car_count = rest_stop_dic[id_rest].get("car", 0)
        truck_count = rest_stop_dic[id_rest].get("truck", 0)

    if row["type"] == "car":
        car_count += 1
    elif row["type"] == "truck":
        truck_count += 1

    rest_stop_dic[id_rest] = {"car": car_count, "truck": truck_count}

In [45]:
rest_stop_dic

{'lon_10.2703215_lat_53.3301407': {'car': 1, 'truck': 0},
 'lon_10.2712414_lat_53.331074': {'car': 1, 'truck': 1},
 'lon_12.140788_lat_52.2431929': {'car': 1, 'truck': 0},
 'lon_12.149456_lat_52.2449316': {'car': 1, 'truck': 0},
 'lon_9.323161_lat_54.0686721': {'car': 1, 'truck': 0},
 'lon_7.5846163_lat_53.2588511': {'car': 2, 'truck': 1},
 'lon_7.2005154_lat_52.2816225': {'car': 1, 'truck': 1},
 'lon_11.2481831_lat_52.1991827': {'car': 1, 'truck': 1},
 'lon_11.2477836_lat_52.1980946': {'car': 1, 'truck': 0},
 'lon_10.7306355_lat_50.559048': {'car': 1, 'truck': 1},
 'lon_10.7298729_lat_50.5605634': {'car': 1, 'truck': 1},
 'lon_13.9503161_lat_52.3112817': {'car': 1, 'truck': 1},
 'lon_13.9496292_lat_52.3118174': {'car': 1, 'truck': 1},
 'lon_10.444298_lat_50.9932399': {'car': 5, 'truck': 1},
 'lon_10.4367549_lat_50.9933128': {'car': 5, 'truck': 1},
 'lon_11.6645768_lat_50.8773612': {'car': 3, 'truck': 0},
 'lon_10.4530068_lat_50.4711034': {'car': 1, 'truck': 2},
 'lon_10.1665917_lat_52

In [46]:
# Length of reststop dictionary
len(rest_stop_dic)

467

## Save geoJSONs

In [ ]:
# Specify the path where you want to save the GeoJSON file
output_file_all_parkings = "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/all_parkings.geojson"
output_file_rest_stations = "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/rest_stations.geojson"
output_file_car_parkings = "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/car_parkings.geojson"
output_file_truck_parkings = "/content/drive/MyDrive/Master Thesis/01 Data Acquisition/OSM GeoJSON/truck_parkings.geojson"

# Export the GeoPandas DataFrame to GeoJSON
#all_parkings.to_file(output_file_all_parkings, driver='GeoJSON')
#rest_stations.to_file(output_file_rest_stations, driver='GeoJSON')
#car_parkings_joined.to_file(output_file_car_parkings, driver='GeoJSON')
#truck_parkings_joined.to_file(output_file_truck_parkings, driver='GeoJSON')